### import

In [12]:
import pandas as pd
import numpy as np
import os
from os.path import join
from datetime import datetime
import pickle
import re
from tqdm import tqdm
pwd = r"D:\1.전산,개발\2.전산관리\DB관리\법조치 수정\통합사건번호"
event_dtype = {'채무자키':str, '계좌키':str, '법조치키':str, '관할법원코드':str, '타채무자키':str,'타법조치키':str}   
#pd.set_option('display.max_rows',None)

### 법원목록 불러오기
- 이제 안 해도 됨(사건검색의 법원목록과 제증명의 법원목록이 다르니 유의)

In [35]:
# from selenium.webdriver.chrome.service import Service
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# import time
# service = Service(executable_path="./파일/chromedriver.exe")
# driver = webdriver.Chrome(service=service)
# driver.get("https://ecfs.scourt.go.kr/ecf/index.jsp")
# time.sleep(2)
# driver.switch_to.frame("FrameSaFind")
# result = driver.find_elements(By.XPATH, '//*[@id="sch_bub_nm"]//option')
# court_list = []
# for v in result:
#     court_list.append(v.text)
# court_list

### 관할법원 수정하기

In [2]:
### 검색 조건 만들기 #######################################
# 목포지원 무안군법원 > 목포로 먼저 가버린다. (검색우선순위를 3가지로 세분화해서 해결함 : 1.군,시법원 2.지원 3.지방법원)
###########################################################
# 법원 정리한 목록 불러오기
법원목록 = r"D:\1.전산,개발\2.전산관리\DB관리\법조치 수정\법원 목록 정리.xlsx"
df_reg = pd.read_excel(법원목록, sheet_name='법원목록').fillna("")
print(len(df_reg))
# dict만들기
cnt = 0
reg1, reg2, reg3, regF1, regF2 = {}, {}, {}, {}, {}
for i, v in df_reg.iterrows() :
    if v.비고 == "가정" : # 지법에는 아무 값도 없음
        if v.시군 != "" :
            regF1[v.시군] = v.법원
        if v.지원 != "" :
            regF2[v.지원] = v.법원
    else :
        if v.시군 != "" :  
            reg1[v.시군] = v.법원
        if v.지원 != "" :
            reg2[v.지원] = v.법원
        if v.지법 != "" :
            reg3[v.지법] = v.법원
print(len(reg1), len(reg2), len(reg3), len(regF1), len(regF2))

206
124 45 14 22 8


In [13]:
### 수정할 파일 불러오기
filename = "솔림_0630_raw"                          #####
df_court_event = pd.read_excel(join(pwd, filename+".xlsx"), dtype=event_dtype).fillna("")#[["법조치키","관할법원","사건번호"]]

In [15]:
# 읽은 파일 바로 pkl로 저장하기
df_court_event.to_pickle(join(pwd, filename+".pkl"))

In [71]:
# # 범위 제한하기
# df_court_event = df_court_event[~df_court_event.관할법원.str.contains("경찰|검찰|법무|법률|세무|차량|구청(?!도)|공사|자산|미입력")]
# df_court_event = df_court_event[df_court_event.관할법원 != ""]
# print(len(df_court_event))

148551


In [51]:
# # 체크하기
# res = df_court_event[(df_court_event["관할법원"].str.contains("광주고")) & (df_court_event["관할법원"].str.contains("목포"))]
# res

,법조치키,경검,관할법원,수정관할법원,확인여부,실명칭부합,관할법원수정여부,사건번호,수정사건번호,사건번호수정여부


In [9]:
for i, v in tqdm(df_court_event.iterrows(), total=len(df_court_event)) :
    out_for_continue = False    ########################### no Touch

    # 검찰, 경찰 등 제외하기 # 사건번호, 대상자명도 수정해야 하니 전체목록을 대상으로 해야한다. 따라서 
    if re.search("경찰|검찰|법무|법률|세무|차량|구청(?!도)|공사|자산|미입력|지급|카드|보증", v.관할법원) or (v.관할법원 == ""):  # 대'구청'도군법원유의
        df_court_event.loc[i, "수정관할법원"] = v.관할법원
        df_court_event.loc[i, "관할법원수정여부"] = "pass"
        continue
        
    # 가정 법원인지 확인
    if re.search("가정", v.관할법원) :
        for exp, right_court_name in regF1.items() :
            if re.search(exp, v.관할법원) :
                df_court_event.loc[i, "수정관할법원"] = right_court_name
                out_for_continue = True
                break
        if out_for_continue : continue # regF1에서 수정할 법원 나왔으면 새로운 반복문 시작
        # regF2 확인하기
        for exp, right_court_name in regF2.items() :
            if re.search(exp, v.관할법원) :
                df_court_event.loc[i, "수정관할법원"] = right_court_name
                out_for_continue = True
                break

        if out_for_continue == False : # '가정법원'에 들어왔음에도, 조건에 걸린 게 없어서 수정한 바가 없음
            df_court_event.loc[i, "수정관할법원"] = v.관할법원
            df_court_event.loc[i, "관할법원수정여부"] = "x"
        continue # 가정법원에 걸렸으니 무조건 여기서 끝냄

    else :
        # 가정법원 아닌 경우 : reg1먼저 확인하기 
        for exp, right_court_name in reg1.items() : # 
            if re.search(",", exp) :
                compile_items = exp.split(",")
                if re.search(compile_items[0], v.관할법원) and re.search(compile_items[1], v.관할법원) : #항상 2개
                    df_court_event.loc[i, "수정관할법원"] = right_court_name
                    out_for_continue = True
                    break
            else :
                if re.search(exp, v.관할법원) :
                    df_court_event.loc[i, "수정관할법원"] = right_court_name
                    out_for_continue = True
                    break
        if out_for_continue : continue # reg1에서 수정할 법원 나왔으면 새로운 반복문 시작

        # reg2 확인하기
        for exp, right_court_name in reg2.items() :
            if re.search(exp, v.관할법원) :
                df_court_event.loc[i, "수정관할법원"] = right_court_name
                out_for_continue = True
                break
        if out_for_continue : continue # reg1에서 수정할 법원 나왔으면 새로운 반복문 시작

        # reg3 확인하기
        for exp, right_court_name in reg3.items() :
            if re.search(exp, v.관할법원) :
                df_court_event.loc[i, "수정관할법원"] = right_court_name
                out_for_continue = True
                break

        if out_for_continue ==False: # 여기까지 왔는데도 바뀐 게 없다면 ..
            df_court_event.loc[i, "수정관할법원"] = v.관할법원
            df_court_event.loc[i, "관할법원수정여부"] = "x"
        continue # reg1에서 수정할 법원 나왔으면 새로운 반복문 시작

print("작업이 완료되었습니다.")

  0%|          | 0/195263 [00:00<?, ?it/s]

100%|██████████| 195263/195263 [01:46<00:00, 1831.27it/s]

작업이 완료되었습니다.


In [10]:
# 엑셀파일 저장
df_court_event.to_excel(join(pwd, filename+"_관할법원수정.xlsx"), index=False)

### 사건번호 수정하기

In [23]:
### 수정할 파일 불러오기
# df_court_event = pd.read_excel(join(pwd, "관할_사건_관할법원.xlsx")).fillna("")

In [14]:
p_event=re.compile(r"(\d{2,4})[\s,]?([가-힣]{1,4})[\s-]?(\d+)") # year = [1], sign = [2], sn=[3]

for i, v in tqdm(df_court_event.iterrows(), total=len(df_court_event)) :
    res = p_event.search(v.사건번호)

    if res != None :
        year = res[1]

        # 1) 순번 앞에 0이 있으면 지우기
        sn = res[3]
        if re.search("[1-9][0-9]*",res[3]) :
            sn = re.search("[1-9][0-9]*",res[3])[0]

        # 2) 공백 지우기
        event = year + res[2] + sn

        # 3) 연도 수정하기 연도 4자리고, 전후가 동일하면 pass, 수정여부에 x
        if len(year) == 4 :
            df_court_event.loc[i,"수정사건번호"] = event
        # 연도 2자리고 00 ~ 23사이면 20 붙이고, 아니면 그대로두기(19는 사건번호에는 안 붙음. 전산조회시 19 있으나 없으나 결과 같음>붙여도 되고 안 붙여도 되고), 수정여부에 ㅇ
        elif len(year) == 2 :
            if int(year) <= 23 :
                df_court_event.loc[i,"수정사건번호"] = "20" + event
            elif int(year) > 70  :
                df.loc[i,"수정사건번호"] = event # 원래 19는 안 붙여도 됨. 검색대상에 19가 붙어있는 경우는 제거 하고 검색해야함 # pass 하지말것.
            else : 
                df_court_event.loc[i,"수정사건번호"] = "연도이상"
        elif len(year) == 3 :
            if int(year) >= 200 and int(year) <=223 :
                df_court_event.loc[i,"수정사건번호"] = event[0] + "0" + event[1:]
            else : 
                df_court_event.loc[i,"수정사건번호"] = "연도이상"
        else :
            df_court_event.loc[i,"수정사건번호"] = "연도이상"
    else :
        df_court_event.loc[i,"수정사건번호"] = v.사건번호

100%|██████████| 195263/195263 [00:13<00:00, 14403.31it/s]


In [15]:
# 사건번호 수정여부 작성하기
df_court_event['사건번호수정여부'] = np.where(df_court_event['사건번호'] != df_court_event['수정사건번호'], 'O', 'X')

In [16]:
df_court_event['사건번호수정여부'].value_counts()

X    195242
O        21
Name: 사건번호수정여부, dtype: int64

In [ ]:
# 확인없이 저장??
df_court_event.사건번호 = df_court_event.수정사건번호

In [11]:
# 전체 저장하기
df_court_event.to_excel(join(pwd, filename + "_사건번호수정.xlsx"), index=False)

In [17]:
# 수정된 것만 저장하기
result_fixed = df_court_event[df_court_event['사건번호수정여부']=="O"]
result_fixed.to_excel(join(pwd, filename + "_사건번호수정된것만.xlsx"), index=False)

In [ ]:
# # 엑셀파일 저장
# df_court_event.to_excel(join(pwd, "관할_사건_사건번호.xlsx"), index=False)

### 관할법원, 사건번호 수정된 것만 별도 파일로 만들기

In [14]:
df_court_event.columns

Index(['법조치키', '관할법원', '사건번호', '수정사건번호', '사건번호수정여부'], dtype='object')

In [7]:
for i, v in tqdm(df_court_event.iterrows(), total=len(df_court_event)) :
    if (v.관할법원 == v.수정관할법원) and (v.사건번호 == v.수정사건번호) : # 둘중 하나도 수정되지 않음
        df_court_event.loc[i,"수정"] = False
    else : # 둘 중 하나라도 수정됨
        df_court_event.loc[i,"수정"] = True        

100%|██████████| 153883/153883 [00:11<00:00, 13335.69it/s]


In [8]:
upload = df_court_event[df_court_event.수정]
print(len(upload))

33310


In [9]:
upload = upload[["법조치키","관할법원",'사건번호','수정관할법원','수정사건번호','수정','관할법원수정여부','사건번호수정여부']]

In [10]:
###########################################################################
upload.to_excel(join(pwd, "result_법원_사건번호_20230125_2차.xlsx"), index=False)
###########################################################################

### 대상자명 수정

In [20]:
### 수정할 파일 불러오기
df_court_event = pd.read_excel(join(pwd, "관할_사건_대상자명.xlsx")).fillna("")

In [21]:
names = tuple(df_court_event.대상자명)
len(names)

153552

In [18]:
name = "윤 정,장태선"
new_name = re.sub(r"주식회사|유한회사|법인|(의\s?)?대표|(의\s?)?(대표)?\s?청산인|(의\s?)?(?<!신용)보증인?|망\b|(의\s?)?(상속인|승계인)|\b차주\b|채무자|형제|자매|\b[주유합자]\b|\([주유합자]|[주유합자]\)|외\s?\d.*", "", name)#이름이 주식인 사람 있음..
new_name = re.sub(r"[^가-힣a-zA-Z]", " ", new_name).strip() #기호가 앞에 나오면 지우질 않음
new_name

'윤 정 장태선'

In [24]:
res, isfixed, whatfixed = [], [], []
for name in tqdm(names, total = len(names)) : 
    
    new_name = re.sub(r"주식회사|유한회사|법인|(의\s?)?대표|(의\s?)?(대표)?\s?청산인|(의\s?)?(?<!신용)보증인?|\b망\b|(의\s?)?(상속인|승계인)|\b차주\b|채무자|형제|자매|\b[주유합자]\b|\([주유합자]|[주유합자]\)|외\s?\d.*", "", name)#이름이 주식인 사람 있음..
    new_name = re.sub(r"[^가-힣a-zA-Z]", " ", new_name).strip() #기호가 앞에 나오면 지우질 않음
    
    spaceSplitList = new_name.split(" ") # 기호는 공백으로 바꿨으니 공백기준

    if len(spaceSplitList[0]) >= 2 : # 공백분리 후 첫 요소의 글자수가 2개 이상인 것
        resultName = spaceSplitList[0]
    # 합쳐야 하는 것 ex) 윤 정 장태선 > 윤정
    elif len(spaceSplitList)>0 : # 2글자가 안 되는데 합칠 것도 없는 경우
        i = 0
        resultName = ""
        while len(spaceSplitList[i])<2 : # 글자수 하나인 경우 계속 합치기
            resultName += spaceSplitList[i]
            i += 1
            if i >= len(spaceSplitList) : break
    else : # 빈값
        resultName = new_name


    # 결과가 2글자가 안 되는 경우 : 원래 대상자명 그대로..
    if len(resultName) < 2 :
        resultName = name

    res.append(resultName)
    isfixed.append(len(name) - len(resultName))
    whatfixed.append(re.sub(resultName, "", name))

df_court_event["수정대상자명"] = res
df_court_event["수정여부"] = isfixed
df_court_event["제거명칭"] = whatfixed

df_court_event.to_excel(join(pwd, "관할_사건_대상자명.xlsx"), index=False)

100%|██████████| 153552/153552 [00:01<00:00, 102359.52it/s]


In [ ]:
df_court_event[df_court_event["대상자명"].str.contains(r"회사")]


In [ ]:
# 엑셀파일 저장
df_court_event.to_excel(join(pwd, "관할_사건_대상자명.xlsx"), index=False)